# Segment 2: Activation Maximization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](#)

**In Segment 1**, we fed images into a CNN and asked: *"What does each neuron respond to?"*
We saw feature maps light up for edges, textures, and shapes.

Now we flip the question entirely:

> **What input image would make a specific neuron fire as hard as possible?**

This is **activation maximization** — and the answers are surprisingly beautiful.

**What you will learn:**
1. What activation-maximized images look like for real neurons inside InceptionV1
2. How gradient ascent on *pixels* (not weights) generates these images
3. Why image parameterization is the difference between noise and clarity
4. How this technique reveals the full feature hierarchy of a network

**Tools:** PyTorch, [Lucent](https://github.com/greentfrapp/lucent) library, InceptionV1

Let's start with the payoff.

In [ ]:
# ---------- Setup ----------

# Lucent is the PyTorch port of Google's Lucid interpretability library.

# Uncomment the line below if running in Colab:

# !pip install torch-lucent



import torch

import numpy as np

import matplotlib.pyplot as plt



from lucent.modelzoo import inceptionv1

from lucent.optvis import render, param, transform, objectives



# Load pretrained InceptionV1 (trained on ImageNet)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = inceptionv1(pretrained=True).to(device).eval()



print(f"Device: {device}")

print(f"Model: InceptionV1 (pretrained on ImageNet)")

print(f"Target layer: mixed4a — a mid-depth Inception module with rich, interpretable features")

In [ ]:
# ---------- The Payoff: what do 10 neurons "dream" about? ----------

# Each image below answers: "What input would make this neuron happiest?"



LAYER = "mixed4a"

NUM_NEURONS = 10



images = []

for i in range(NUM_NEURONS):

    # objectives.channel targets one channel (neuron) in the named layer

    # render_vis optimizes a random image to maximize that neuron's activation

    imgs = render.render_vis(

        model,

        objectives.channel(LAYER, i),

        param_f=lambda: param.image(128, fft=True, decorrelate=True),

        thresholds=(512,),

        show_inline=False,

        show_image=False,

    )

    images.append(imgs[0][0])   # extract the H x W x 3 numpy array

    print(f"  Neuron {i} done")



# --- Display as a 2x5 gallery ---

fig, axes = plt.subplots(2, 5, figsize=(16, 7))

for i, ax in enumerate(axes.flat):

    ax.imshow(images[i])

    ax.set_title(f"mixed4a : {i}", fontsize=11, fontweight="bold")

    ax.axis("off")



fig.suptitle(

    '10 Neurons, 10 Different "Dreams" — each neuron has learned to detect a unique pattern',

    fontsize=13, fontweight="bold",

)

plt.tight_layout()

plt.show()

## What just happened?

Each image above was **not drawn by a human** and **not taken by a camera**. It was generated by an optimization process:

1. Start with a random noise image
2. Feed it through the frozen network
3. Measure how strongly the target neuron activates
4. Compute the gradient of that activation with respect to the input pixels
5. Nudge the pixels in the direction that *increases* the activation (gradient ascent)
6. Repeat for hundreds of steps

The result reveals the **visual pattern that neuron has learned to detect** — textures, curves, grids, color combinations, or parts of objects.

Take a moment to look at the gallery above. Some images might resemble fur, honeycombs, spirals, or woven fabric. Each one is a different feature detector that InceptionV1 discovered from millions of training images.

**But how does this optimization actually work under the hood?** Let's build it from scratch.

In [ ]:
# ---------- DIY: activation maximization from scratch ----------

# We'll implement the core algorithm in raw PyTorch (no Lucent)

# to understand exactly what's happening.



TARGET_NEURON = 0  # same neuron we visualized above in the gallery



# Step 1: Start from random noise — requires_grad=True because we optimize pixels

input_img = torch.randn(1, 3, 224, 224, device=device, requires_grad=True)

optimizer = torch.optim.Adam([input_img], lr=0.05)



# Step 2: Hook into mixed4a to read its output during forward pass

activation_store = {}

def hook_fn(module, inp, out):

    activation_store["value"] = out



hook_handle = dict(model.named_modules())["mixed4a"].register_forward_hook(hook_fn)



# Step 3: Gradient ascent — maximize the target channel's mean activation

NUM_STEPS = 256

act_history = []           # track activation value at each step

snapshots = []             # save the image at key moments

snapshot_steps = {0, 32, 64, 128, 255}



for step in range(NUM_STEPS):

    optimizer.zero_grad()

    model(input_img)



    # Mean activation of the target channel across all spatial positions

    act_val = activation_store["value"][0, TARGET_NEURON].mean()

    (-act_val).backward()  # negate because Adam minimizes, but we want to maximize

    optimizer.step()



    act_history.append(act_val.item())



    if step in snapshot_steps:

        snap = input_img[0].detach().cpu().permute(1, 2, 0).numpy()

        snap = (snap - snap.min()) / (snap.max() - snap.min() + 1e-8)

        snapshots.append((step, snap.copy()))



hook_handle.remove()



# --- Display: filmstrip of snapshots + activation curve ---

fig = plt.figure(figsize=(16, 4))



for i, (step, snap) in enumerate(snapshots):

    ax = fig.add_subplot(1, len(snapshots) + 1, i + 1)

    ax.imshow(snap)

    ax.set_title(f"Step {step}", fontsize=9)

    ax.axis("off")



ax_curve = fig.add_subplot(1, len(snapshots) + 1, len(snapshots) + 1)

ax_curve.plot(act_history, color="steelblue", linewidth=1.5)

ax_curve.set_xlabel("Step")

ax_curve.set_ylabel("Activation")

ax_curve.set_title("Neuron activation\nover optimization", fontsize=9)



fig.suptitle(

    f"DIY gradient ascent on mixed4a neuron {TARGET_NEURON} — the pattern emerges, but it's noisy",

    fontsize=12, fontweight="bold",

)

plt.tight_layout()

plt.show()



print("The structure is there, but the image is full of high-frequency noise.")

print("This is where parameterization tricks make all the difference.")

## Why so noisy?

Our DIY version optimized **raw pixel values** directly. The optimizer found high-frequency patterns that technically increase the neuron's activation but look like static to our eyes. This is a known problem — unrestricted pixel optimization produces adversarial-looking images.

Lucent solves this with two key tricks:

- **FFT parameterization:** Represent the image in the frequency domain. This naturally penalizes high frequencies, so the optimizer produces smoother, more natural patterns.

- **Color decorrelation:** Optimize in a decorrelated color space (based on natural image statistics). This lets the optimizer explore realistic color combinations instead of getting stuck in unnatural hues.

Let's see the difference — same neuron, same number of steps, three levels of sophistication.

In [ ]:
# ---------- Same neuron, three approaches ----------

target = objectives.channel(LAYER, TARGET_NEURON)



# (A) Lucent with naive pixel parameterization (no tricks)

imgs_naive = render.render_vis(

    model, target,

    param_f=lambda: param.image(128, fft=False, decorrelate=False),

    transforms=[],

    thresholds=(512,),

    show_inline=False, show_image=False,

)



# (B) Lucent with full pipeline (FFT + decorrelation + standard transforms)

imgs_full = render.render_vis(

    model, target,

    param_f=lambda: param.image(128, fft=True, decorrelate=True),

    transforms=transform.standard_transforms,

    thresholds=(512,),

    show_inline=False, show_image=False,

)



# --- Three-panel comparison ---

fig, axes = plt.subplots(1, 3, figsize=(15, 5))



# Panel 1: Our DIY result from the previous cell

diy_final = snapshots[-1][1]

axes[0].imshow(diy_final)

axes[0].set_title("Our DIY loop\n(raw pixels, no tricks)", fontsize=11)



# Panel 2: Lucent without tricks

axes[1].imshow(imgs_naive[0][0])

axes[1].set_title("Lucent — naive pixels\n(fft=False, decorrelate=False)", fontsize=11)



# Panel 3: Lucent with full pipeline

axes[2].imshow(imgs_full[0][0])

axes[2].set_title("Lucent — full pipeline\n(fft + decorrelation + transforms)", fontsize=11)



for ax in axes:

    ax.axis("off")



fig.suptitle(

    f"mixed4a neuron {TARGET_NEURON}: parameterization turns noise into clarity",

    fontsize=13, fontweight="bold",

)

plt.tight_layout()

plt.show()



print("Same neuron, same optimization objective — the only difference is how we represent the image.")

## What we know so far

| Step | What we saw |
|------|------------|
| **Gallery** | 10 neurons in mixed4a each learned to detect a unique visual pattern |
| **DIY loop** | Gradient ascent on pixels works, but raw optimization produces noise |
| **Parameterization** | FFT + decorrelation + transforms produce clean, interpretable results |

One question remains: **does this only work for mixed4a?**

Segment 1 showed us that CNNs build a feature hierarchy — early layers detect edges, deeper layers detect objects. Let's apply activation maximization across multiple layers and neurons to see if that hierarchy shows up here too.

In [ ]:
# ---------- Summary figure: 3 neurons across 3 layers ----------

# Rows = different neurons, Columns = increasing network depth



poster_layers = ["mixed3a", "mixed4a", "mixed5b"]

poster_neurons = [0, 5, 9]

col_labels = ["Early-mid (mixed3a)", "Mid (mixed4a)", "Late (mixed5b)"]



fig, axes = plt.subplots(3, 3, figsize=(12, 12))



for row, neuron_idx in enumerate(poster_neurons):

    for col, lyr in enumerate(poster_layers):

        print(f"  {lyr} neuron {neuron_idx}...", end=" ", flush=True)

        imgs = render.render_vis(

            model,

            objectives.channel(lyr, neuron_idx),

            param_f=lambda: param.image(128, fft=True, decorrelate=True),

            thresholds=(512,),

            show_inline=False, show_image=False,

        )

        axes[row, col].imshow(imgs[0][0])

        axes[row, col].axis("off")



        if row == 0:

            axes[row, col].set_title(col_labels[col], fontsize=11, fontweight="bold")



    axes[row, 0].set_ylabel(

        f"Neuron {neuron_idx}", fontsize=11, fontweight="bold",

        rotation=0, labelpad=55, va="center",

    )



print("\ndone")



fig.suptitle(

    "Features grow more complex with depth — the hierarchy Segment 1 predicted",

    fontsize=13, fontweight="bold", y=1.01,

)

plt.tight_layout()

plt.show()

## Summary

| Concept | What we learned |
|---------|----------------|
| **Activation maximization** | Generate images that maximally activate a specific neuron — revealing the pattern it has learned to detect |
| **Gradient ascent on pixels** | Freeze the network, optimize the input — the reverse of training |
| **Parameterization matters** | FFT + color decorrelation + transforms turn noise into interpretable images |
| **Feature hierarchy** | Early layers dream of edges, mid layers of textures, deep layers of object parts |

### Limitations to keep in mind

- These images show what **maximally** excites a neuron, not the full range of inputs it responds to in practice.
- Some neurons are **polysemantic** — they respond to multiple unrelated patterns. A single activation-maximized image cannot reveal this.
- The parameterization choices (FFT, decorrelation) act as a **prior** on what kinds of images we allow. Different priors can yield different-looking results.

---

### What's next

We now know what neurons *want* to see (activation maximization). The natural follow-up is to ask: **what do they *actually* see in real images?** Finding the real-world image patches that most strongly activate specific neurons connects these synthetic dreams back to the visual world — and that's where the story continues.